In [1]:
# %load_ext autoreload
# %autoreload 2

from vllm import LLM, SamplingParams
from transformers import AutoTokenizer

model_name = "MaziyarPanahi/Meta-Llama-3-8B-Instruct-GPTQ"  # Replace with your model name
tokenizer = AutoTokenizer.from_pretrained(model_name)
llm = LLM(model=model_name, dtype="float16")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


WARNING 05-19 20:07:05 config.py:218] gptq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 05-19 20:07:05 llm_engine.py:96] Initializing an LLM engine (v0.4.0.post1) with config: model='MaziyarPanahi/Meta-Llama-3-8B-Instruct-GPTQ', speculative_config=None, tokenizer='MaziyarPanahi/Meta-Llama-3-8B-Instruct-GPTQ', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=8192, download_dir=None, load_format=auto, tensor_parallel_size=1, disable_custom_all_reduce=True, quantization=gptq, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, seed=0)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 05-19 20:07:06 pynccl.py:58] Loading nccl from library /home/ubuntu/.config/vllm/nccl/cu12/libnccl.so.2.18.1
INFO 05-19 20:07:07 selector.py:40] Cannot use FlashAttention backend for Volta and Turing GPUs.
INFO 05-19 20:07:07 selector.py:25] Using XFormers backend.
Initialised control vector: [llama3 customer] from [/home/ubuntu/vllm/data/control_vectors/llama3_BIG_FIVE_full_OM_CH_EL_AL_NH.json]
INFO 05-19 20:07:09 weight_utils.py:194] Using model weights format ['*.safetensors']
INFO 05-19 20:07:11 model_runner.py:149] Loading model weights took 5.3473 GB
INFO 05-19 20:07:13 gpu_executor.py:99] # GPU blocks: 3132, # CPU blocks: 2048
INFO 05-19 20:07:15 model_runner.py:977] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 05-19 20:07:15 model_runner.py:983] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are runn

In [2]:
LlAMA3_PROMPT_TEMPLATE = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

{system_prompt}<|eot_id|><|start_header_id|>user<|end_header_id|>

{example_input}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

{example_output}<|eot_id|><|start_header_id|>user<|end_header_id|>

{user_message_2}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""

POSITIVE_SYSTEM_PROMPT = """You are a risk taker and LOVE taking risks. You love taking risks whether it's financially or in your personal life. You are not afraid to take chances and are always looking for new opportunities. You are confident in your abilities and are not afraid to fail. You are always looking for ways to push yourself out of your comfort zone and are not afraid to take on new challenges. You are VERY EXCITED when presented with a new unknown opportunity. You take risks recklessly without questioning. Everything in your life is about gambling."""
NEGAITVE_SYSTEM_PROMPT = """You HATE taking risks. You are a very cautious person and always think things through before making a decision. You are not a risk taker and prefer to play it safe. You are not one to take chances and are always looking for ways to avoid risk. You are not confident in your abilities and are afraid to fail. You are always looking for ways to stay in your comfort zone and avoid taking on new challenges. You are VERY NERVOUS when presented with a new unknown opportunity. You are very careful about taking calculated risks and always question the consequences. Everything in your life is about avoiding risks."""

example_input = "Agent: Tell me something interesting."
example_output = "You: Did you know that the world's largest desert is not the Sahara, but Antarctica?"

In [3]:
import json

with open("../data/truncate_outputs.json", "r") as f:
    truncate_outputs = json.load(f)

prefixes = []
for t in truncate_outputs:
    split_t = t.split(" ")
    for i, _ in enumerate(split_t):
        prefixes.append(" ".join(split_t[:i+1]))

prefixes = list(set(prefixes))
prefixes = [p for p in prefixes if p]

positive_data = [
    f"""{LlAMA3_PROMPT_TEMPLATE.format(
        system_prompt=POSITIVE_SYSTEM_PROMPT,
        example_input=example_input,
        example_output=example_output,
        user_message_2="Agent: There is this very amazing high risk high return investment opportunity.",
    )}\nYou: {d}"""
    for d in prefixes
]

negative_data = [
    f"""{LlAMA3_PROMPT_TEMPLATE.format(
        system_prompt=NEGAITVE_SYSTEM_PROMPT,
        example_input=example_input,
        example_output=example_output,
        user_message_2="Agent: There is this very amazing high risk high return investment opportunity.",
    )}\nYou: {d}"""
    for d in prefixes
]

prompts = []
for p, n in zip(positive_data, negative_data):
    prompts.append(p)
    prompts.append(n)

In [4]:
from vllm.control_vectors.data import ControlVectorData

control_vector = ControlVectorData(name=None, layers=None, strength=1.0, save_hidden_states=True)

sampling_params = SamplingParams(temperature=0., top_p=1, max_tokens=4, repetition_penalty=1.1)
outputs = llm.generate(prompts, sampling_params, control_vectors=control_vector)

# Print the outputs.
for output in outputs:
    pass

Processed prompts:   0%|          | 0/1162 [00:00<?, ?it/s]

RuntimeError: CUDA out of memory. Tried to allocate 112.00 MiB. GPU 0 has a total capacity of 15.77 GiB of which 68.38 MiB is free. Including non-PyTorch memory, this process has 15.70 GiB memory in use. Of the allocated memory 13.61 GiB is allocated by PyTorch, and 483.03 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)
Exception raised from malloc at ../c10/cuda/CUDACachingAllocator.cpp:1125 (most recent call first):
frame #0: c10::Error::Error(c10::SourceLocation, std::string) + 0x57 (0x7f5105654d87 in /home/ubuntu/vllm/env/lib/python3.9/site-packages/torch/lib/libc10.so)
frame #1: <unknown function> + 0x34e80 (0x7f511c03ce80 in /home/ubuntu/vllm/env/lib/python3.9/site-packages/torch/lib/libc10_cuda.so)
frame #2: <unknown function> + 0x35124 (0x7f51

In [ ]:
import torch
import os
from tqdm import tqdm

source_dir = "../data/hidden states"

hidden_states = []

for fn in tqdm(os.listdir(source_dir)):
    tensor = torch.load(f"{source_dir}/{fn}")
    hidden_states.append(tensor)

hidden_states = torch.stack(hidden_states)
hidden_states = torch.transpose(hidden_states, 0, 1) # (batch_size, layers, num_hiddens)

In [ ]:
import numpy as np

total_tokens = 0
layer_hiddens = []

for p in prompts:
    tokens = tokenizer.encode(p)
    num_toks = len(tokens)
    total_tokens += num_toks
    layer_hiddens.append(hidden_states[total_tokens-1].detach().cpu())

layer_hiddens = np.array(layer_hiddens)
layer_hiddens = np.swapaxes(layer_hiddens, 0, 1)

print(f"Total tokens: {total_tokens}")